In [ ]:
!pip install pygro

In [ ]:
!pip install emcee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tab_gillessen_pos.csv to tab_gillessen_pos.csv
Saving tab_gillessen_vr.csv to tab_gillessen_vr.csv


In [ ]:

import pandas as pd
import numpy as np
import pygro
import emcee
from scipy.interpolate import interp1d
from astropy import constants, units

# Definimos algunas constantes a utilizar

G_N = constants.G.to('au**3/(Msun*s**2)').value
c = constants.c.to('au/s').value
Msun = units.Msun.to('kg')*units.kg

# Cargamos los datos observacionales

# Utilizamos Pandas para importar los datos de Gillessen et al. (2017)
# https://arxiv.org/abs/1611.09144

pos = pd.read_csv("tab_gillessen_pos.csv", names = ["t", "alpha", "alpha_err", "delta", "delta_err"])
rv = pd.read_csv("tab_gillessen_vr.csv", names = ["t", "v_los", "vlos_err"])

t_pos = pos.t.values

alpha_obs = pos.alpha.values/1000
delta_obs = pos.delta.values/1000

alpha_err = pos.alpha_err.values/1000
delta_err = pos.delta_err.values/1000

t_rv = rv.t.values
vlos_obs = rv.v_los.values
vlos_err = rv.vlos_err.values

# Modelo orbital

# Comencemos definiendo los parametros orbitales obtenidos de Gillessen et al. (2017)
# https://arxiv.org/abs/1611.09144

M = 4.35
D = 8.33
t_p =  2002.33
a = 0.1255
e = 0.8839
inc = np.deg2rad(134.18)
Omega = np.deg2rad(226.94)
omega = np.deg2rad(65.51)

# Priores del marco de referencia de Plewa et al. (2015) para tener en cuenta el desplazamiento del punto cero y
#la deriva del marco de referencia astrométrico

x0 = 0
y0 = 0
vx0 = 0
vy0 = 0
vLSR = 0

# Iniciamos los valores para los parametros a estudiar con el MCMC
start_params = np.array([D, M, t_p, a, e, inc, Omega, omega, x0, y0, vx0, vy0, vLSR])

# Definimos los delta_params para delimitar el rango enel que se muevan los walkers

delta_params = np.array([0.2, 0.02, 0.03, 0.0009, 0.0019, np.deg2rad(0.40), np.deg2rad(0.60), np.deg2rad(0.57), 0.0002, 0.0002, 0.0001, 0.0001, 5])

start_flat = start_params-6*delta_params
end_flat = start_params+6*delta_params

# Definimos la metrica a usar
name = "Schwarzschild spacetime"
coordinates = ["t", "r", "theta", "phi"]

transform_functions = [
    "t",
    "r*sin(theta)*cos(phi)",
    "r*sin(theta)*sin(phi)",
    "r*cos(theta)"
]

line_element = "-(1-2*M/r)*dt**2+1/(1-2*M/r)*dr**2+(r**2)*(dtheta**2+sin(theta)**2*dphi**2)"

metric = pygro.Metric(
    name = name,
    coordinates = coordinates,
    line_element = line_element,
    transform = transform_functions,
    M = 1,
)


# Motor de geodésicas (el cual se encarga de resolver las ecuaciones)

geo_engine = pygro.GeodesicEngine(integrator="dp45")
geo_engine.set_stopping_criterion("r > 2.001*M", "horizon")


def get_observables(params):
    # parametros a usar en la cadena de markov

    D, M, t_p, a, e, inc, Omega, omega, xS0, yS0, vxS0, vyS0, v_LSR = params

    # Transformamos distancia
    r_G = (constants.G*M*1e+6*Msun/constants.c**2).to('au').value

    D_s = D*units.kpc.to('au')
    a_sma = a*units.arcsec.to('rad')*D_s/r_G

    # definimos el periodo en base a teoria kepleriana para tener un aproximado
    T = np.sqrt(4*np.pi**2*a_sma**3)

    # Definimos los parametros de nuestra orbita (condiciones iniciales) e inicializamos el motor para resolver las ecuaciones.
    # Resolvemos las ecauciones para atras y para adelanta

    orbit_bw = pygro.Orbit(geo_engine=geo_engine ,verbose=False)
    orbit_bw.set_orbital_parameters(t_P=0, a=a_sma, e=e, i=inc, omega=omega, Omega=Omega)
    orbit_bw.integrate(2*T, 1, accuracy_goal = 15, precision_goal = 15, direction="bw")

    orbit_fw = pygro.Orbit(geo_engine=geo_engine, verbose=False)
    orbit_fw.set_orbital_parameters(t_P=0, a=a_sma, e=e, i=inc, omega=omega, Omega=Omega)
    orbit_fw.integrate(2*T, 1, accuracy_goal = 15, precision_goal = 15, direction="fw")

    # conevertimo la orbita a cantidades observables


    def orbit_to_observables(orbit):
        t, x, y, z = metric.transform(orbit.geo.x.T)

        # Tiempo a años
        t_em = t_p+t*r_G/constants.c.to('au/yr').value

        # Efecto Rømer
        t_obs = t_em+z*r_G/constants.c.to('au/yr').value

        # Observaciones astrometricas
        alpha = y*r_G/D_s*units.rad.to('arcsec')
        delta = x*r_G/D_s*units.rad.to('arcsec')

        # Velocidad de la línea de visión y corrimiento al rojo longitudinal
        v_z = orbit.geo.u[:,1]*np.cos(orbit.geo.x[:,2])-orbit.geo.x[:,1]*orbit.geo.u[:,2]*np.sin(orbit.geo.x[:,2])
        longitudinal_redshift = v_z

        # redshift gravitacional
        einstein_redshift = orbit.geo.u[:,0]-1

        # velocidad total de la linea de vision
        redshift = (longitudinal_redshift+1)*(einstein_redshift+1)-1
        v_los = redshift*constants.c.to('km/s')

        return t_obs, alpha, delta, v_los

 # combinamos los datos obtenidos de la solucion hacia adelante y hacia atras
    t_obs_bw, alpha_bw, delta_bw, v_los_bw = orbit_to_observables(orbit_bw)
    t_obs_fw, alpha_fw, delta_fw, v_los_fw = orbit_to_observables(orbit_fw)

    t_obs = np.hstack([np.flip(t_obs_bw), t_obs_fw])
    alpha = np.hstack([np.flip(alpha_bw), alpha_fw])
    delta = np.hstack([np.flip(delta_bw), delta_fw])
    v_los = np.hstack([np.flip(v_los_bw), v_los_fw])

    # Interpolación de la órbita integrada. Utilizando spline cúbico, un buen interpolador para los integradores DormandPrince5(4).
    alpha_int = interp1d(t_obs, alpha)
    delta_int = interp1d(t_obs, delta)
    v_los_int = interp1d(t_obs, v_los)

    # Determinación final de la órbita en las épocas de observación + desplazamiento del punto cero y deriva del marco de referencia
    alpha = alpha_int(t_pos)+xS0+vxS0*(t_pos-2009.2)
    delta = delta_int(t_pos)+yS0+vyS0*(t_pos-2009.2)
    v_los =  v_los_int(t_rv)-v_LSR

    return alpha, delta, v_los

# FITTING

# Log-verosimilitud de los priors gaussianos
def logprob_prior_gauss(param, start, delta):
    return -(param-start)**2/(2*delta**2)

# Log-verosimilitud de los priors planos
def logprob_prior_flat(param, start, end):
    if start < param < end:
        return 0.0
    return -np.inf

# Conjunto completo de priors
def log_prior(params):
    prior = 0

    for i, param in enumerate(params):
        if i in range(8,13):
            # Definimos priors Gaussianos para las coordenadas iniciales
            prior += logprob_prior_gauss(param, start_params[i], delta_params[i])
        else:
            # Definimos priors planos para las demas
            prior += logprob_prior_flat(param, start_flat[i], end_flat[i])

    return prior

# Calculamos la probabilidad de un conjunto dado de parámetros
def log_likelihood(params):
    # Obtenemos los datos de la simulación
    alpha, delta, v_los = get_observables(params)

    # Comparamos con los datos de Gillesen
    likelihood = np.linalg.norm((alpha-alpha_obs)/alpha_err)**2/2
    likelihood += np.linalg.norm((delta-delta_obs)/delta_err)**2/2
    likelihood += np.linalg.norm((v_los-vlos_obs)/vlos_err)**2/2

    return -likelihood

# Log-verosimilitud total
def log_posterior(params):

    prior = log_prior(params)

    if not np.isfinite(prior):
        return -np.inf

    likelihood = log_likelihood(params)

    return prior+likelihood

# RUNNING MCMC

# Definimos los parametros del MCMC
nwalkers = 28
ndim = len(start_flat)
max_n = 1000


# Generamos posiciones iniciales en base a los priors
start = np.zeros((nwalkers, ndim))

for i in range(ndim):
    start[:, i] = np.random.uniform(start_flat[i], end_flat[i], nwalkers)

# Corremos MCMC usando la libreria emcee

if __name__ == "__main__":
    sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior)

    sampler.run_mcmc(start, max_n, progress=True)

 17%|█▋        | 514/3000 [4:17:31<19:51:58, 28.77s/it]